In [1]:
pip install cassandra-driver

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
from cassandra.cluster import Cluster

# Connect to the local Cassandra instance
cluster = Cluster(['127.0.0.1'])  # Assuming Cassandra is running on localhost
session = cluster.connect()



In [14]:
# session.execute("CREATE KEYSPACE IF NOT EXISTS bigdata WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 3}")
session.set_keyspace("bigdata")

In [5]:
# Execute your Cassandra commands here
query = "CREATE TABLE IF NOT EXISTS test (id UUID PRIMARY KEY, amt FLOAT)"
session.execute(query)



In [8]:
from cassandra.cluster import Cluster
import json
# JSON data to insert
json_data = {
    "id": "123e4567-e89b-12d3-a456-426614174000",
    "amt": 100.50
}

# Convert JSON data to a string
json_string = json.dumps(json_data)

# Prepare and execute the INSERT statement with JSON data
insert_query = "INSERT INTO test JSON %s"
session.execute(insert_query, [json_string])

# Close the connection
# session.shutdown()
# cluster.shutdown()


In [9]:
# Prepare and execute the SELECT statement
select_query = "SELECT * FROM test"
result_set = session.execute(select_query)

# Process the result set
for row in result_set:
    print(row)


Row(id=UUID('123e4567-e89b-12d3-a456-426614174000'), amt=100.5)


In [ ]:
import csv
from cassandra.cluster import Cluster
import json
cluster = Cluster()
session = cluster.connect()
session.set_keyspace("bigdata")
session.execute("CREATE TABLE IF NOT EXISTS training (txn_id varchar PRIMARY KEY,cc_no bigint ,txn_time varchar, year smallint, month smallint, day smallint, hour smallint, minute smallint,amount float, cc_provider int, merchant varchar,merchant_index smallint, country_name varchar, country_code varchar,country_index smallint, age int, marital_status varchar,marital_status_index smallint,gender varchar,gender_index smallint,loan varchar,loan_index smallint, status varchar,status_index smallint)")
with open('50_50.csv', 'r',encoding="ISO-8859-1") as csvfile:
        reader=csv.reader(csvfile)
        for row in reader:
        	#print(row)
        	#msg=json.loads(row)
        	#print(msg)
        	dic={
        	"txn_id":row[0],
        	"cc_no":row[1],
        	"txn_time":row[2],
        	"year":row[3],
        	"month":row[4],
        	"day":row[5],
        	"hour":row[6],
        	"minute":row[7],
        	"amount":row[8],
        	"cc_provider":row[9],
        	"merchant":row[10],
            "merchant_index":row[11],
        	"country_name":row[12],
        	"country_code":row[13],
            "country_index":row[14],
        	"age":row[15],
        	"marital_status":row[16],
            "marital_status_index":row[17],
        	"gender":row[18],
            "gender_index":row[19],
        	"loan":row[20],
            "loan_index":row[21],
        	"status":row[22],
            "status_index":row[23]
        	}            
        	r = json.dumps(dic)
        	#print(r)
        	strq=" INSERT INTO training JSON "+"'"+r+"'"
        	session.execute(strq)

In [19]:
from cassandra.cluster import Cluster

# # Connect to Cassandra
# cluster = Cluster(['127.0.0.1'])  # Change to your Cassandra host
# session = cluster.connect('bigdata')  # Change to your keyspace

# Define CQL query to create a table
create_table_query = """
    CREATE TABLE transaction_data (
    trans_date_trans_time text,
    cc_num bigint,
    merchant text,
    category text,
    amt decimal,
    first text,
    last text,
    gender text,
    street text,
    city text,
    state text,
    zip int,
    lat decimal,
    long decimal,
    city_pop int,
    job text,
    dob date,
    trans_num text,
    unix_time bigint,
    merch_lat decimal,
    merch_long decimal,
    merchant_label int,
    category_label int,
    gender_label int,
    job_label int,
    is_fraud int,
    is_fraud_prediction int,
    PRIMARY KEY (trans_num)
    
);

"""
drop_query = 'drop table transaction_data'
# Execute the create table query
session.execute(create_table_query)

# # Close the connection
# session.shutdown()
# cluster.shutdown()


In [29]:
########### testing bank csv
import csv
import json
import datetime
import time
from cassandra.cluster import Cluster

def insert_data_from_csv(csv_file, keyspace, table):
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for i, row in enumerate(reader):
            if i >= 100:
                break
            # Convert row to JSON format
            row['ts'] = int(time.time() * 1000)
            json_row = json.dumps(row)
            # Define CQL query to insert JSON data into Cassandra
            insert_query = f"INSERT INTO {keyspace}.{table} JSON '{json_row}';"
            # Execute the insert query
            session.execute(insert_query)

# Specify CSV file path, keyspace, and table name
csv_file_path = 'bank.csv'
keyspace_name = 'bigdata'
table_name = 'bank'

# Call the function to insert data from CSV into Cassandra
insert_data_from_csv(csv_file_path, keyspace_name, table_name)

# # Close the connection
# session.shutdown()
# cluster.shutdown()


In [32]:
import pandas as pd

def insert_data(df, keyspace, table):
    for row in df.to_dict(orient='records'):
        json_row = json.dumps(row)
        # Define CQL query to insert JSON data into Cassandra
        insert_query = f"INSERT INTO {keyspace}.{table} JSON '{json_row}';"
        # Execute the insert query
        session.execute(insert_query)

# Specify CSV file path, keyspace, and table name
df = pd.read_csv('bank.csv')
keyspace_name = 'bigdata'
table_name = 'bank'

for i in range(20):
    random_rows = df.sample(n=20)
    random_rows['ts'] = int(time.time() * 1000)
    insert_data(random_rows, keyspace_name, table_name)
    time.sleep(5)
# # Close the connection
# session.shutdown()
# cluster.shutdown()

KeyboardInterrupt: 

In [32]:
query = "SELECT * FROM transaction_data;"
rows = session.execute(query)
import pandas as pd
pd.DataFrame(rows)

,trans_num,amt,category,category_label,cc_num,city,city_pop,dob,first,gender,gender_label,is_fraud,is_fraud_prediction,job,job_label,last,lat,long,merch_lat,merch_long,merchant,merchant_label,state,street,trans_date_trans_time,unix_time,zip
0,2e373e0cd6cecdc4415af26d522ca42d,68.66,grocery_net,3,180094419304907,Sun City,54287,1972-09-22,Sharon,F,0,0,0,"Designer, exhibition/display",122,Smith,33.7467,-117.1721,33.750727000000005,-116.415818,fraud_Hills-Olson,255,CA,17666 David Valleys,2019-08-01 03:21:37,1343791297,92585
1,5151c60669d29fb73ec3690489e020f8,33.56,misc_pos,9,4481131401752,Shrewsbury,35299,1975-04-30,Frank,M,1,0,0,English as a second language teacher,178,Foster,42.2848,-71.7205,42.740677000000005,-72.59328599999999,"fraud_Wintheiser, Dietrich and Schimmel",673,MA,37910 Ward Lights,2019-04-30 15:08:10,1335798490,1545
2,b0ce2e69b71c535e2fd68e2f4e0bd327,46.61,food_dining,1,4191109180926792,Amanda,4512,1992-10-03,Jennifer,F,0,0,0,Farm manager,191,Spencer,39.6251,-82.7552,39.729005,-81.782448,fraud_Jast and Sons,283,OH,477 Wheeler Burg,2019-05-27 15:41:49,1338133309,43102
3,fc29e7c11f1d84dfd14f995b81b1cf9a,63.76,gas_transport,2,4561368699336875,Clifton,167,1988-11-01,Pamela,F,0,0,0,Museum/gallery conservator,303,Lewis,34.9906,-81.8327,35.768583,-82.06145500000001,fraud_Emard Inc,157,SC,653 Jessica Extension Apt. 668,2020-07-18 06:18:56,1374128336,29324
4,a9dd902c9f0aabd11730743a7add3bf9,24.49,entertainment,0,6011477612335392,Maria Stein,2274,1943-12-17,Anthony,M,1,0,0,"Designer, television/film set",128,Roberts,40.4062,-84.5076,39.503889,-84.030238,"fraud_Cormier, Stracke and Thiel",106,OH,95488 Cabrera Well,2020-07-23 17:29:49,1374600589,45860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,20a4d6a9d102f9a92f0665601289f9dd,9.57,kids_pets,7,372246459334925,Deadwood,1979,1970-03-13,Rick,M,1,0,0,Multimedia programmer,301,Martinez,44.3566,-103.6999,44.143838,-103.733337,fraud_Hoppe-Parisian,264,SD,062 Poole Hollow Apt. 815,2019-11-03 12:18:56,1351945136,57732
57,0eb6de47059e64e18887588a3a378aa1,6.63,kids_pets,7,4653879239169997,Smiths Grove,6841,1999-06-06,Monica,F,0,0,0,"Therapist, sports",473,Tucker,37.0581,-86.1938,37.727108,-86.961391,fraud_Windler LLC,671,KY,302 Christina Islands,2020-12-22 14:08:47,1387721327,42171
58,22d20fb0da83b4c2bec9fffcbc71e404,87.85,food_dining,1,4956828990005111019,Elizabeth,124967,1980-12-21,Kenneth,M,1,0,0,Operational researcher,321,Robinson,40.6747,-74.2239,41.000643,-75.092575,fraud_Leannon-Ward,378,NJ,269 Sanchez Rapids,2019-12-06 14:20:11,1354803611,7208
59,8d2807844fceccad027f5a2918da0209,114.28,shopping_pos,12,3597337756918966,Glen Rock,7565,1952-03-08,Linda,F,0,0,0,Insurance claims handler,249,Gonzalez,39.7813,-76.7477,38.810682,-76.496011,"fraud_Watsica, Haag and Considine",656,PA,17667 Price Spur Apt. 366,2020-10-08 11:33:46,1381232026,17327


In [27]:
pd.set_option('display.max_column', None)

In [ ]:
# Close the connection
session.shutdown()
cluster.shutdown()